In [1]:
# two kinds of double pendulums

In [3]:
import numpy as np
from numpy import sin, cos
import cv2
import imageio

##### Initial parameters #####
size = [900, 1300]
white = [255, 255, 255]
black = [0, 0, 0]

m1 = 20
m2 = 20
r1 = 150
r2 = 150
a1 = np.pi / 6
a2 = np.pi / 3
a1_v = 0
a2_v = 0
g = 1
endPoints = []

img = np.zeros(size)


def acceleration1():
    exp1 = -g * (2 * m1 + m2) * sin(a1)
    exp2 = -m2 * g * sin(a1 - 2 * a2)
    exp3 = -2 * sin(a1 - a2) * m2
    exp4 = (a2_v*a2_v) * r2 + (a1_v*a1_v) * r1 * cos(a1 - a2)
    den = r1 * (2 * m1 + m2 - m2 * cos(2 * a1 - 2 * a2))
    a1_a = (exp1 + exp2 + exp3 * exp4) / den
    return a1_a

def acceleration2():
    exp1 = 2 * sin(a1 - a2)
    exp2 = (a1_v*a1_v) * r1 * (m1 + m2)
    exp3 = g * (m1 + m2) * cos(a1)
    exp4 = (a2_v*a2_v) * r2 * m2 * cos(a1 - a2)
    den = r2 * (2 * m1 + m2 - m2 * cos(2 * a1 - 2 * a2))
    a2_a = (exp1 * (exp2 + exp3 + exp4)) / den
    return a2_a


def trace(coords):
    for i in range(len(coords) - 1):
        cv2.line(img, (int(coords[i][0]), int(coords[i][1])),
                 (int(coords[i+1][0]), int(coords[i+1][1])), black, 1)

count=0
while True:
    cv2.imshow('Double Pendulum', img)
    if cv2.waitKey(23) & 0xFF == ord('q') or count>500:
        break

    a1_a = acceleration1()
    a2_a = acceleration2()

    x1 = r1 * sin(a1) + size[1] / 2
    y1 = r1 * cos(a1) + size[0] / 5

    x2 = x1 + r2 * sin(a2)
    y2 = y1 + r2 * cos(a2)

    endPoints.append((x2, y2))

    img.fill(255)
    cv2.line(img, (int(size[1]/2), int(size[0]/5)), (int(x1), int(y1)), black, 2)
    cv2.circle(img, (int(x1), int(y1)), m1, black, -1)
    cv2.line(img, (int(x1), int(y1)), (int(x2), int(y2)), black, 2)
    cv2.circle(img, (int(x2), int(y2)), m2, black, -1)

    trace(endPoints)

    a1_v += a1_a
    a2_v += a2_a
    a1 += a1_v
    a2 += a2_v
    
    count+=1



In [4]:
# Repeat: Antiphase synchronization of two nonidentical pendulums
# attention: not repeated successfully

###### Initial parameters ######

wait_time=100
# x
x = 100
x_a = 0
x_v = 0
# theta
a1 = 0.05#np.pi / 6
a2 = -0.05#np.pi / 3
a1_v = 0
a2_v = 0
# parameters
lambda_1= 0.01
m1 = 20
m2 = m1
M = 50
l1 = 1
l2 = 1
eta = 3*(0.1)**4
f1=0.35
f2=0.35
# plot
B = 250
L = 800
#tao = (B/2)*(np.sqrt(l/g))/(M+2m)
tao = 0.8
# mu = (b/2)*(np.sqrt(l/g))
mu = 0.02

###### plot
size = [1000, 1000]
img = np.zeros(size)


##### update derivation
def acceleration_x():
    x_a = (-2*tao*x_v-mu*(F1+F2))/(1-mu*(cos(a1)**2+cos(a2)**2))
    return x_a
def acceleration1():
    a1_a = -(2*eta*a1_v+(sin(a1)+x_a*cos(a1))/(l1)-f1)
    return a1_a
def acceleration2():
    a2_a = -(2*eta*a2_v+(sin(a2)+x_a*cos(a2))/(l2)-f2)
    return a2_a
def F_1(F):
    return l1*(cos(a1)*(f1*np.sign(F)-2*eta*a1_v-sin(a1)/l1)-a1_v**2*sin(a1))
def F_2(F):
    return l2*(cos(a2)*(f2*np.sign(F)-2*eta*a2_v-sin(a2)/l2)-a2_v**2*sin(a2))
def frame_to_gif(frame_list):
    gif = imageio.mimsave('./result.gif', frame_list, 'GIF', duration=0.00085)


###### compute F1 F2
F1=F_1(1)
F2=F_2(1)


###### start loop
frames = []
count = 0
while True:
    # compute derivative parameters
    a1_a = acceleration1()
    a2_a = acceleration2()
    x_a = acceleration_x()
    # fix the position of ball
    x1 = l1*100 * sin(a1) + L/3+x
    y1 = l1*100 * cos(a1) + B

    x2 = l2*100 * sin(a2) + 2*L/3+x
    y2 = l2*100 * cos(a2) + B
        
    # update paramters
    a1_v += a1_a*lambda_1
    a2_v += a2_a*lambda_1
    a1 += a1_v
    a2 += a2_v
        
    x_v += x_a*lambda_1
    x +=x_v
    
    F1=F_1(-1*np.sign(a1))
    F2=F_2(-1*np.sign(a2))
    
    # cv2
    img = np.zeros((size[0], size[1], 3), dtype=np.uint8)
    cv2.rectangle(img,(int(x),5),(int(x+L),B),[0, 255, 0],cv2.FILLED)
    cv2.line(img, (int(L / 3+x), B), (int(x1), int(y1)), [255, 255, 255], 2, cv2.LINE_AA)
    cv2.circle(img, (int(x1), int(y1)), m1, [0,0,255], cv2.FILLED, cv2.LINE_4)
    cv2.line(img, (int(2*L/3+x), int(B)), (int(x2), int(y2)), [255, 255, 255], 2, cv2.LINE_AA)
    cv2.circle(img, (int(x2), int(y2)), m2, [255, 0, 0], cv2.FILLED, cv2.LINE_4)
    
    # save imgs
    cv2.imwrite('imgs/result_'+str(count)+'.jpg', img)
    
    count+=1
    # show img
    cv2.imshow('Double Pendulum', img)
    cv2.waitKey(wait_time)
    # maximum 500 frames
    if (cv2.waitKey(10) & 0xFF == ord('q')) or (count>500):
        break
    

frames = []
for i in range(500):
    frames.append(cv2.imread('imgs/result_'+str(i)+'.jpg'))
frame_to_gif(frames)